In [1]:
import sqlite3
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
db = sqlite3.connect('L20DB_demo.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

In [2]:
with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

In [3]:
with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

In [4]:
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

In [5]:
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
query = '''SELECT * FROM candidates'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,16,Mike,Huckabee,,R
1,20,Barack,Obama,,D
2,22,Rudolph,Giuliani,,R
3,24,Mike,Gravel,,D
4,26,John,Edwards,,D
5,29,Bill,Richardson,,D
6,30,Duncan,Hunter,,R
7,31,Dennis,Kucinich,,D
8,32,Ron,Paul,,R
9,33,Joseph,Biden,,D


In [6]:
query = '''SELECT * FROM candidates WHERE middle_init <> ""'''
viz_tables(candidate_cols, query)

,id,first_name,last_name,middle_init,party
0,34,Hillary,Clinton,R.,D
1,39,Christopher,Dodd,J.,D
2,41,Fred,Thompson,D.,R


In [7]:
print("{} candidates have a middle initial.".format(viz_tables(candidate_cols, query).shape[0]))

3 candidates have a middle initial.


### Display the contributors where the state is "PA"


In [8]:
contributors_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
query = '''SELECT * FROM contributors WHERE state="PA"'''
viz_tables(contributors_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,71,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
1,72,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
2,94,Raught,Philip,M,4714 Plum Way,,Pittsburgh,PA,15201,-1046.0,2008-04-21,32
3,95,Ferrara,Judith,D,1508 Waterford Road,,Yardley,PA,19067,-1100.0,2008-04-21,32
4,166,ABEL,JOHN,H.,422 THOMAS STREET,,BETHLEHEM,PA,180153316,200.0,2008-01-22,37


### Display the contributors where the amount contributed is greater than  $1000.00 .

In [9]:
query = '''SELECT * FROM contributors WHERE amount > 1000'''
viz_tables(contributors_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
1,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16
2,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16
5,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20
6,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20
7,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.0,2007-09-30,20
8,46,Buchanan,John,,2025 NW 29th Rd,,Boca Raton,FL,334316303,1300.0,2007-08-09,20
9,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35


### Display the contributors from "UT" where the amount contributed is greater than  $1000.00 .

In [10]:
query = '''SELECT * FROM contributors WHERE state="UT" and amount > 1000'''
viz_tables(contributors_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20
1,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20


### Display the contributors who didn't list their state

In [11]:
query = '''SELECT * FROM contributors WHERE (state is null or state="")'''
viz_tables(contributors_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,126,BOURNE,TRAVIS,,LAGE KAART 77,,BRASSCHATT,,2930,-500.0,2008-11-20,35


### Display the contributors from "WA" and "PA"

In [12]:
query = '''SELECT * FROM contributors WHERE state IN ("WA", "PA")'''
viz_tables(contributors_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,63,BURKE,SUZANNE,M.,3401 EVANSTON,,SEATTLE,WA,981038677,-700.0,2008-03-05,22
1,71,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
2,72,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
3,94,Raught,Philip,M,4714 Plum Way,,Pittsburgh,PA,15201,-1046.0,2008-04-21,32
4,95,Ferrara,Judith,D,1508 Waterford Road,,Yardley,PA,19067,-1100.0,2008-04-21,32
5,101,Aaronson,Rebecca,,2000 Village Green Dr Apt 12,,Mill Creek,WA,980125787,100.0,2008-02-08,34
6,107,Aaronson,Rebecca,,2000 Village Green Dr Apt 12,,Mill Creek,WA,980125787,100.0,2008-02-14,34
7,166,ABEL,JOHN,H.,422 THOMAS STREET,,BETHLEHEM,PA,180153316,200.0,2008-01-22,37


### Display the contributors who contributed between  \$100.00 and  \$200.00 .

In [13]:
query = '''SELECT * FROM contributors WHERE amount BETWEEN 100.00 and 200.00'''
viz_tables(contributors_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,4,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,100.0,2007-06-21,16
1,5,Akin,Charles,,10187 Sugar Creek Road,,Bentonville,AR,72712,100.0,2007-06-16,16
2,13,Allison,Rebecca,,3206 Summit Court,,Little Rock,AR,72227,200.0,2007-06-12,16
3,18,Arbogast,Robert,,12900 State Route 56 SE,,Mount Sterling,OH,43143,100.0,2007-06-22,16
4,28,Buckheit,Bruce,,8904 KAREN DR,,FAIRFAX,VA,220312731,100.0,2007-09-19,20
5,32,Buck,Thomas,,4206 Terrace Street,,Kansas City,MO,64111,100.0,2007-09-25,20
6,33,Buck,Jay,K.,1855 Old Willow Rd Unit 322,,Northfield,IL,600932918,200.0,2007-09-12,20
7,38,Bucher,Ida,M,1400 Warnall Ave,,Los Angeles,CA,900245333,100.0,2007-07-10,20
8,47,Buchanan,John,,2025 NW 29th Rd,,Boca Raton,FL,334316303,200.0,2007-08-14,20
9,101,Aaronson,Rebecca,,2000 Village Green Dr Apt 12,,Mill Creek,WA,980125787,100.0,2008-02-08,34


## Step 3

### Sort the candidates table by last_name.

In [14]:
query = '''SELECT * FROM contributors ORDER BY last_name'''
viz_tables(contributors_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,163,AARONS,CHARLES,,1730 SHORE DRIVE,,ANCHORAGE,AK,995153207,300.00,2008-01-30,37
1,164,AARONS,CHARLES,,1730 SHORE DRIVE,,ANCHORAGE,AK,995153207,410.00,2008-01-15,37
2,165,AARONS,CHARLES,,1730 SHORE DRIVE,,ANCHORAGE,AK,995153207,500.00,2008-01-09,37
3,162,ABACHERLI,SHIRLEY,M.,29875 NEWPORT ROAD,,MENIFEE,CA,925849524,150.00,2008-01-28,37
4,161,ABAIR,PETER,,40 EVANS STREET,,WATERTOWN,MA,24722150,25.00,2008-01-09,37
5,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,,MIAMI SPRINGS,FL,331663832,2600.00,2008-01-25,37
6,159,ABBO,PAULINE,MORENCY,10720 JACOB LANE,,WHITE LAKE,MI,483862274,35.00,2008-01-07,37
7,158,ABBOT,DAVID,M.,56 SALEM STREET,,ANDOVER,MA,18102114,200.00,2008-01-21,37
8,147,ABBOTT,WELDON,S.,777 EAST SOUTH TEMPLE 4E,,SALT LAKE CITY,UT,841021269,100.00,2007-09-29,35
9,148,ABBOTT,WELDON,S.,777 EAST SOUTH TEMPLE 4E,,SALT LAKE CITY,UT,841021269,50.00,2007-08-09,35


### Sort by the amount in decending order where amount is restricted to be between  $1000.00  and  $5000.00 .

In [15]:
query = '''SELECT * FROM contributors WHERE amount BETWEEN 1000.00 and 5000.00 ORDER BY amount DESC'''
viz_tables(contributors_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20
1,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
2,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16
5,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20
6,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.0,2007-09-30,20
7,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35
8,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
9,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16


### Sort the contributors who donted between  $1000.00  and  $5000.00  by candidate_id and then by amount in descending order.

In [16]:
query = '''SELECT * FROM contributors WHERE amount BETWEEN 1000.00 and 5000.00 ORDER BY candidate_id, amount DESC'''
viz_tables(contributors_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16
1,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16
2,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16
3,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
4,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16
5,9,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1000.0,2007-06-11,16
6,11,Allison,John W.,,P.O. Box 1089,,Conway,AR,72033,1000.0,2007-05-18,16
7,12,Allison,Rebecca,,3206 Summit Court,,Little Rock,AR,72227,1000.0,2007-04-25,16
8,20,Atiq,Omar,,7200 S Hazel Street,,Pine Bluff,AR,71603,1000.0,2007-05-18,16
9,21,Atiq,Omar,,7200 S Hazel Street,,Pine Bluff,AR,71603,1000.0,2007-06-27,16


## Step 4

### Get the first and last name of contributors. Make sure each row has distinct values.

In [17]:
query = '''SELECT DISTINCT first_name, last_name FROM contributors '''
viz_tables(['first_name', 'last_name'], query)

,first_name,last_name
0,Steven,Agee
1,Don,Ahrens
2,Charles,Akin
3,Mike,Akin
4,Rebecca,Akin
5,Brittni,Aldridge
6,John D.,Allen
7,John W.,Allison
8,Rebecca,Allison
9,R.D.,Altes


## Step 5

### Add a new column to the contributors table called full_name. The value in that column should be in the form last_name, first_name.

In [19]:
contributors_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
query = '''SELECT id, last_name, first_name FROM contributors'''
full_name_and_id = [(attr[1] + ", " + attr[2], attr[0]) for attr in cursor.execute(query).fetchall()] # List of tuples: (full_name, id)
update = '''UPDATE contributors SET full_name = ? WHERE id = ?'''
for rows in full_name_and_id:
    cursor.execute(update, rows)
query = '''SELECT * FROM contributors'''
viz_tables(contributors_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,full_name
0,1,Agee,Steven,,549 Laurel Branch Road,,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,2,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,3,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,4,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,5,Akin,Charles,,10187 Sugar Creek Road,,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"
5,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16,"Akin, Mike"
6,7,Akin,Rebecca,,181 Baywood Lane,,Monticello,AR,71655,500.0,2007-05-18,16,"Akin, Rebecca"
7,8,Aldridge,Brittni,,"808 Capitol Square Place, SW",,Washington,DC,20024,250.0,2007-06-06,16,"Aldridge, Brittni"
8,9,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1000.0,2007-06-11,16,"Allen, John D."
9,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16,"Allen, John D."


### Change the value in the full_name column to the string "Too Much" if someone donated more than  $1000.00 .

In [20]:
contributors_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
query = '''SELECT id, last_name, first_name FROM contributors'''
full_name_and_id = [(attr[1] + ", " + attr[2], attr[0]) for attr in cursor.execute(query).fetchall()] # List of tuples: (full_name, id)
update = '''UPDATE contributors SET full_name = ? WHERE id = ?'''
for rows in full_name_and_id:
    cursor.execute(update, rows)
query = '''SELECT * FROM contributors'''
update = '''UPDATE contributors SET full_name = "Too Much" WHERE amount > 1000.00'''
cursor.execute(update)

viz_tables(contributors_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,full_name
0,1,Agee,Steven,,549 Laurel Branch Road,,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,2,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,3,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,4,Ahrens,Don,,4034 Rennellwood Way,,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,5,Akin,Charles,,10187 Sugar Creek Road,,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"
5,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16,Too Much
6,7,Akin,Rebecca,,181 Baywood Lane,,Monticello,AR,71655,500.0,2007-05-18,16,"Akin, Rebecca"
7,8,Aldridge,Brittni,,"808 Capitol Square Place, SW",,Washington,DC,20024,250.0,2007-06-06,16,"Aldridge, Brittni"
8,9,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1000.0,2007-06-11,16,"Allen, John D."
9,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16,Too Much


## Step 6

###  Count how many donations there were above  $1000.00 .

In [21]:
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
function = '''SELECT COUNT(amount) AS count FROM contributors WHERE amount > 1000.00'''
viz_tables(['count'], function)

,count
0,12


### Calculate the average donation.

In [22]:
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
function = '''SELECT AVG(amount) FROM contributors'''
viz_tables(['average'], function)

,average
0,3.418114


### Calculate the average contribution from each state and display in a table.

In [23]:
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]
function = '''SELECT state, AVG(amount) FROM contributors GROUP BY state'''

viz_tables(['state', 'average amount'], function)

,state,average amount
0,,-500.000000
1,AK,403.333333
2,AR,1183.333333
3,AZ,120.000000
4,CA,-217.988261
5,CO,-1455.750000
6,CT,2300.000000
7,DC,-309.982000
8,FL,-135.000000
9,IA,250.000000
